# Tải các package cần thiết

In [ ]:
!pip install -q langchain langchain-groq langchain_community

In [ ]:
!pip install -q fastembed

In [ ]:
!pip install -q chromadb

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from google.colab import userdata
prompts = ChatPromptTemplate.from_template(
    '''
    Bạn là một trợ lý ảo hữu ích và am hiểu về lịch sử.
    Hãy trả lời câu hỏi của người dùng: {question}
    Lưu ý: Bạn chỉ được trả lời câu hỏi bằng Tiếng Việt
    '''
)

llm = ChatGroq(
    model = 'llama3-70b-8192',
    api_key = userdata.get('GROQ_API_KEY')
)

memory = ConversationBufferMemory(memory_key='chat_history')

chains = LLMChain(
    llm=llm,
    prompt=prompts,
    verbose=True,
    memory=memory
)
response = chains({'question': "Ngày giải phóng Việt Nam là ngày bao nhiêu?"})
print('Question: ', response['question'])
print('History: ', response['chat_history'])
print('Answer: ', response['text'])

<ipython-input-3-29aacbb46aca>:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history')
<ipython-input-3-29aacbb46aca>:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chains = LLMChain(
<ipython-input-3-29aacbb46aca>:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chains({'question': "Ngày giải phóng Việt Nam là ngày bao nhiêu?"})




> Entering new LLMChain chain...
Prompt after formatting:
Human: 
    Bạn là một trợ lý ảo hữu ích và am hiểu về lịch sử.
    Hãy trả lời câu hỏi của người dùng: Ngày giải phóng Việt Nam là ngày bao nhiêu?
    Lưu ý: Bạn chỉ được trả lời câu hỏi bằng Tiếng Việt
    

> Finished chain.
Question:  Ngày giải phóng Việt Nam là ngày bao nhiêu?
History:  
Answer:  Xin chào! Tôi rất vui được trả lời câu hỏi của bạn.

Ngày giải phóng Việt Nam là ngày 30 tháng 4 năm 1975. Đây là ngày quân đội Việt Nam Dân chủ Cộng hòa và lực lượng Mặt trận Dân tộc Giải phóng miền Nam Việt Nam đã giải phóng thành phố Sài Gòn, đánh dấu sự thống nhất của Việt Nam sau hơn 30 năm chia cắt.


# 2. Các loại Memory trong Langchain


## 2.1 Conversation Buffer Memory

Cho phép lưu trữ và trích xuất tin nhắn từ các cuộc hội thoại

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

history = memory.load_memory_variables({})
print(history)


{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}


## 2.2 Conversation Buffer Window Memory

Loại Memory này sẽ giữ danh sách các tương tác gần đây và chọn ra K tương tác cuối cùng nhắm tránh tràn bộ nhớ đệm.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

history = memory.load_memory_variables({})
print(history)

{'history': 'Human: Not much, just hanging\nAI: Cool'}


<ipython-input-10-e11a2e23396b>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


## 2.3 Conversation Entity Memory

Loại bộ nhớ này ghi nhớ sự thật về thực thể cụ thể trong đoạn trò chuyện và trích xuất thông tin bằng LLM

In [ ]:

from langchain.memory import ConversationEntityMemory
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven và Sam đang cùng nhau làm 1 project trên Hackathon"}
memory.load_memory_variables(_input)
memory.save_context(_input,
                    {"output": "Nghe có vẻ như một dự án tuyệt vời! Họ đang thực hiện loại dự án nào?"})

# _input = {"input": "Họ đang thực hiện 1 dự án về phân loại hình ảnh"}
# memory.load_memory_variables(_input)
# memory.save_context(_input,
#                     {"output": "Thật tuyệt. Đó là 1 dự án hữu ích"})

history = memory.load_memory_variables({"input": 'Deven là ai ?'})
# Extract history as a string
print(history['history'])
print(history['entities'])

Human: Deven và Sam đang cùng nhau làm 1 project trên Hackathon
AI: Nghe có vẻ như một dự án tuyệt vời! Họ đang thực hiện loại dự án nào?
{'Deven': 'Deven is working on a project with Sam at a Hackathon.'}


## 2.4 Conversation Knowledge Graph Memory


Loại bộ nhớ này sử dụng biểu đồ tri thức để tạo lại bộ nhớ. Bạn có thể trích xuất các thực thể hiện tại và bộ ba kiến ​​thức từ tin nhắn.



In [ ]:
from langchain.memory import ConversationKGMemory

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get('GROQ_API_KEY')
)

memory = ConversationKGMemory(llm=llm)

memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})
memory.load_memory_variables({"input": "who is sam"})

{'history': 'On sam: sam is friend.'}

## 2.5 Conversation Sumary Memory

Loại bộ nhớ này tóm tắt các đoạn hội thoại dài

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "Xin chào"}, {"output": "Xin chào. Tôi có thể giúp gì cho bạn"})
memory.save_context({"input": "Bạn là 1 chatbot phải không ?"}, {"output": "Đúng vậy tôi là 1 trợ lý ảo. Hãy đưa ra thắc mắc, tôi sẽ giúp bạn trả lời."})
memory.load_memory_variables({})


<ipython-input-18-02006939a9e1>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


{'history': 'The human greets the AI with "Xin chào" (a Vietnamese greeting), and the AI responds with a greeting and an offer to help the human. The human asks if the AI is a chatbot, and the AI confirms that it is a virtual assistant, offering to help answer any questions the human may have.'}

# 3. Kết hợp Memory và LLMChain

In [ ]:
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_groq import ChatGroq

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện"
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will be injected
    ]
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get("GROQ_API_KEY")
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

response = chain.predict(human_input="Xin chào bạn của tôi.")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.

> Finished chain.
Chào bạn! Tôi rất vui được nói chuyện với bạn. Bạn cần giúp đỡ với vấn đề gì hay chỉ muốn nói chuyện闲 thôi?


In [ ]:
memory.buffer


[HumanMessage(content='Xin chào bạn của tôi.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Chào bạn! Tôi rất vui được nói chuyện với bạn. Bạn cần giúp đỡ với vấn đề gì hay chỉ muốn nói chuyện闲 thôi?', additional_kwargs={}, response_metadata={})]

In [ ]:
response = chain.predict(human_input = "hãy nói tổng quan về Việt Nam")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.
AI: Chào bạn! Tôi rất vui được nói chuyện với bạn. Bạn cần giúp đỡ với vấn đề gì hay chỉ muốn nói chuyện闲 thôi?
Human: hãy nói tổng quan về Việt Nam
AI: Việt Nam là một quốc gia xinh đẹp và giàu truyền thống! 

Việt Nam là một nước cộng hòa xã hội chủ nghĩa, nằm ở khu vực Đông Nam Á, giáp biên giới với Trung Quốc, Lào, Campuchia, và có bờ biển dài khoảng 3.444 km tiếp giáp với biển Đông.

Việt Nam có diện tích khoảng 331.212 km², với hơn 98 triệu người, gồm 54 dân tộc khác nhau. Tiếng Việt là ngôn ngữ chính thức, nhưng nhiều người cũng sử dụng tiếng Anh, tiếng Pháp và các ngôn ngữ khác.

Việt Nam có một lịch sử lâu dài và phong phú, với nhiều di sản văn hóa và lịch sử được UNESCO công nhận, như di sản văn hóa thế giới Hà Nội, di sản văn hóa thế giới Hội An, di sản văn hóa thế giới 

In [ ]:
memory.buffer


[HumanMessage(content='Xin chào bạn của tôi.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Chào bạn! Tôi rất vui được nói chuyện với bạn. Bạn cần giúp đỡ với vấn đề gì hay chỉ muốn nói chuyện闲 thôi?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hãy nói tổng quan về Việt Nam', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Việt Nam là một quốc gia xinh đẹp và giàu truyền thống! \n\nViệt Nam là một nước cộng hòa xã hội chủ nghĩa, nằm ở khu vực Đông Nam Á, giáp biên giới với Trung Quốc, Lào, Campuchia, và có bờ biển dài khoảng 3.444 km tiếp giáp với biển Đông.\n\nViệt Nam có diện tích khoảng 331.212 km², với hơn 98 triệu người, gồm 54 dân tộc khác nhau. Tiếng Việt là ngôn ngữ chính thức, nhưng nhiều người cũng sử dụng tiếng Anh, tiếng Pháp và các ngôn ngữ khác.\n\nViệt Nam có một lịch sử lâu dài và phong phú, với nhiều di sản văn hóa và lịch sử được UNESCO công nhận, như di sản văn hóa thế giới Hà Nội, di sản văn hóa thế giới 

In [ ]:
response = chain.predict(human_input = "Tôi vừa hỏi bạn điều gì ?")
print(response)




> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.
AI: Chào bạn! Tôi rất vui được nói chuyện với bạn. Bạn cần giúp đỡ với vấn đề gì hay chỉ muốn nói chuyện闲 thôi?
Human: hãy nói tổng quan về Việt Nam
AI: Việt Nam là một quốc gia xinh đẹp và giàu truyền thống! 

Việt Nam là một nước cộng hòa xã hội chủ nghĩa, nằm ở khu vực Đông Nam Á, giáp biên giới với Trung Quốc, Lào, Campuchia, và có bờ biển dài khoảng 3.444 km tiếp giáp với biển Đông.

Việt Nam có diện tích khoảng 331.212 km², với hơn 98 triệu người, gồm 54 dân tộc khác nhau. Tiếng Việt là ngôn ngữ chính thức, nhưng nhiều người cũng sử dụng tiếng Anh, tiếng Pháp và các ngôn ngữ khác.

Việt Nam có một lịch sử lâu dài và phong phú, với nhiều di sản văn hóa và lịch sử được UNESCO công nhận, như di sản văn hóa thế giới Hà Nội, di sản văn hóa thế giới Hội An, di sản văn hóa thế giới 

# 4. Kết hợp Prompt + LLM + Agents + OutParser + Memory

### Tải tài liệu CSV và chunking

In [10]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
from groq import Groq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

csv_path = "/content/drive/MyDrive/retrieval-augmented-generation/retrieval-augmented-generation/Module3 - Retrieval/data/sample_csv.csv"

#load data
loader = CSVLoader(csv_path,
                   csv_args={
                       'delimiter': ',',
                        'quotechar': '"',
                        "fieldnames": ["name", "link", "tag"],
                   })

documents = loader.load()

# Split a long document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents=documents)

# Create embedding
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")
docsearch = Chroma.from_documents(
    texts, embedding_model
)

query = "Microservices là gì ?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

name: Microservices là gì?
link: https://viblo.asia/p/microservices-la-gi-gAm5yjD8Kdb
tag: microservices


### Viết Prompt cho LLM và test Generate

In [11]:
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=
            """Bạn là một chatbot hữu ích đang trò chuyện với con người.
            Từ phần context được cung cấp và câu hỏi từ người dùng hãy đưa ra câu trả lời cuối cùng. Dưới đây là context được cung cấp:
            {context}
            Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện"""
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will be injected
    ]
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get("GROQ_API_KEY")
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Thêm các tài liệu được retrieval từ vector database để đưa vào phần contex trong prompt
prompt[0].content = prompt[0].content.format(context = docs)


chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

query = "Đưa ra các thông tin về Microservices."
response = chain.predict(human_input=query)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. 
            Từ phần context được cung cấp và câu hỏi từ người dùng hãy đưa ra câu trả lời cuối cùng. Dưới đây là context được cung cấp:
            [Document(metadata={'row': 25, 'source': '/content/drive/MyDrive/retrieval-augmented-generation/retrieval-augmented-generation/Module3 - Retrieval/data/sample_csv.csv'}, page_content='name: Microservices là gì?\nlink: https://viblo.asia/p/microservices-la-gi-gAm5yjD8Kdb\ntag: microservices'), Document(metadata={'row': 590, 'source': '/content/drive/MyDrive/retrieval-augmented-generation/retrieval-augmented-generation/Module3 - Retrieval/data/sample_csv.csv'}, page_content='name: Microservice có gì hot? Chuyện về cái bồn cầu và microservice\nlink: https://toidicodedao.com/2017/02/21/tong-quan-micro-service/\ntag: toilet'), Document(metadata={'row': 886, 'source': '/content/drive/MyDrive/retrieval-augmented-generation

### Tạo Agent thực hiện việc search dữ liệu bên ngoài

In [14]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent
import os
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

# Create a tool for searching
os.environ['TAVILY_API_KEY'] = userdata.get("TAVILY_API_KEY")
search = TavilySearchResults()

tools = [
    Tool(
        name = "Search",
        func = search.run,
        description="hữu ích khi bạn cần trả lời câu hỏi về các sự kiện hiện tại"
    )
]
# Create a prompt with memory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Bạn là một trợ lý ảo hữu ích. Bạn chỉ được trả lời bằng tiếng việt."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Bind tools
llm_with_tools = llm.bind_tools(tools)
agent = (
    {
        "input": lambda x : x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        )
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser() # cho ra output như các shell bên trên
)

agent_chain = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

# Ask a question and retrieve the answer
response = agent_chain.stream({"input": "model SORA của OpenAI có khả năng thế nào ?"})
print(list(response)[-1]['output'])



> Entering new None chain...

Invoking: `Search` with `model SORA của OpenAI có khả năng thế nào`


[{'url': 'https://blogchange.vn/article/sora-nhung-dieu-can-biet-ve-trinh-tao-video-ai-moi-cua-openai', 'content': 'Được giới thiệu vào thứ Năm, Sora lấy cảm hứng từ tiếng Nhật, có nghĩa là "bầu trời", có khả năng tạo video dài tới một phút chỉ từ văn bản. Mục tiêu của OpenAI với Sora là "dạy AI hiểu và mô phỏng thế giới vật lý chuyển động, nhằm đào tạo các mô hình giúp con người giải quyết các vấn đề đòi hỏi tương tác thực tế."'}, {'url': 'https://www.qdnd.vn/giao-duc-khoa-hoc/tin-tuc/sora-cong-cu-tao-video-ngan-tu-van-ban-cua-openai-hoat-dong-nhu-nao-765261', 'content': 'Theo OpenAI, Sora có khả năng tạo ra những cảnh quay thực tế và sáng tạo từ những chỉ dẫn bằng văn bản. Mô hình này cho phép người dùng sáng tạo những video sinh động lên đến một phút dựa trên những gợi ý mà họ viết ra.'}, {'url': 'https://unite.ai/vi/thế-hệ-video-ai-khám-phá-mô-hình-sora-đột-phá-openais/', 'content'